In [0]:
f = open("cidades_dados_inmet.txt", "r")
str_cidades = f.read()
str_cidades = str_cidades.replace('\n',',')
print(str_cidades)
list_cidades = str_cidades.split(',')
list_cidades

In [0]:
list_cidades = ['ACARAU','AGUA BRANCA','ALAGOINHAS','ALTO PARNAIBA','APODI','ARACAJU','ARCOVERDE','AREIA','BACABAL',
                'BALSAS','BARBALHA','BARRA','BARRA DO CORDA','BARREIRAS','BOM JESUS DA LAPA','BOM JESUS DO PIAUI',
                'CABROBO','CAETITE','CALDEIRAO','CAMPINA GRANDE','CAMPOS SALES','CANAVIEIRAS','CARACOL','CARAVELAS',
                'CARINHANHA','CAROLINA','CAXIAS','CEARA MIRIM','CHAPADINHA','CIPO','COLINAS','CORRENTINA','CRATEUS',
                'CRUZ DAS ALMAS','CRUZETA','ESPERANTINA','FEIRA DE SANTANA','FLORANIA','FLORIANO','FORTALEZA',
                'GARANHUNS','GUARAMIRANGA','GUARATINGA','IGUATU','IMPERATRIZ','IRECE','ITABAIANINHA','ITABERABA',
                'ITIRUCU (JAGUAQUARA)','ITUACU','JACOBINA','JAGUARUANA','JOAO PESSOA','LENCOIS',
                'LUZILANDIA(LAG.DO PIAUI)','MACAU','MACEIO','MONTE SANTO','MONTEIRO','MORADA NOVA','MORRO DO CHAPEU',
                'NATAL','OURICURI','PALMEIRA DOS INDIOS','PAO DE ACUCAR','PARNAIBA','PATOS','PAULISTANA',
                'PAULO AFONSO','PETROLINA','PICOS','PIRIPIRI','PORTO DE PEDRAS','PROPRIA','QUIXERAMOBIM',
                'RECIFE (CURADO)','REMANSO','S?O GONCALO','SALVADOR (ONDINA)','SAO JOAO DO PIAUI','SAO LUIS',
                'SENHOR DO BONFIM','SERIDO (CAICO)','SERRINHA','SOBRAL','STa. R. DE CASSIA (IBIPETUBA)','SURUBIM',
                'TAUA','TERESINA','TRIUNFO','TURIACU','VALE DO GURGUEIA (CRISTIANO CASTRO)','VITORIA DA CONQUISTA',
                'ZE DOCA']

In [0]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (76.0.3809.100-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [0]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from io import StringIO
import pandas as pd
from tqdm import tqdm
import pickle

In [0]:
def get_data_from_str(string):
    s1 = 'Latitude  (graus) : '
    s1_f = '\nLongitude'
    s2 = 'Longitude (graus) : '
    s2_f = '\nAltitude'
    s3 = 'Altitude  (metros): '

    latitude = string[string.find(s1)+len(s1):string.find(s1_f)]
    longitude = string[string.find(s2)+len(s2):string.find(s2_f)]
    altitude = string[string.find(s3)+len(s3):string.find(s3)+len(s3)+5]

    substring_data = string[string.find('Estacao'):string.find('\n</pre>')]
    TESTDATA = StringIO(substring_data)
    df = pd.read_csv(TESTDATA, sep=";")
    dict_llad = {'Latitude':latitude, 'Longitude':longitude, 'Altitude': altitude, 'Dados':df}

    return dict_llad

In [0]:
lista_dados = []

for cidade in tqdm(list_cidades):
    while True:
        try:
            #enter the link to the website you want to automate login.
            website_link="http://www.inmet.gov.br/projetos/rede/pesquisa/inicio.php"
            options = webdriver.ChromeOptions()
            options.add_argument('--headless')
            options.add_argument('--no-sandbox')
            options.add_argument('--disable-dev-shm-usage')

            browser = webdriver.Chrome('chromedriver', options=options)	#for macOS users[for others use chrome vis chromedriver]
            browser.get(website_link)	

            browser.find_element_by_name("mCod").send_keys("hugoabreu1002@gmail.com")	
            browser.find_element_by_name("mSenha").send_keys("5ggv239z")
            browser.find_element_by_xpath('//input[@type="submit" and @value=" Acessar "]').click()
            browser.implicitly_wait(3)

            # print(browser.page_source)

            # page1 = open('page1.html','w')
            # page1.write(browser.page_source)
            # page1.close()

            #--------------------------------------------------------------------#

            browser.find_element_by_xpath('//a[@href="form_mapas_mensal.php"]').click()
            browser.implicitly_wait(3)

            # page2 = open('page2.html','w')
            # page2.write(browser.page_source)
            # page2.close()

            # print(browser.page_source)

            #--------------------------------------------------------------------#

            browser.find_element_by_xpath('//input[@name="mRelDtInicio"]').send_keys('01/01/1999')
            browser.find_element_by_xpath('//input[@name="mRelDtFim"]').send_keys('31/12/2015')
            browser.find_element_by_xpath('//select[@name="mRelRegiao"]/option[text()="Nordeste"]').click()

            for i in range(2, 18):
                browser.find_element_by_xpath('//input[@name="mOpcaoAtribX"]'.replace("X",str(i))).click()

            browser.find_element_by_xpath('//input[@name="btnProcesso1"]').click()
            browser.switch_to.alert.accept(); 
            browser.implicitly_wait(3) 

            #print(browser.page_source)
            # page3 = open('page3.html','w')
            # page3.write(browser.page_source)
            # page3.close()

            #--------------------------------------------------------------------#

            browser.find_element_by_link_text('_'.replace('_', cidade)).click()

            # page4 = open('page4.html','w')
            # page4.write(browser.page_source)
            # page4.close()

            #-------------------------------------------------------------------#

            browser.find_element_by_xpath('//img[@alt="Dados"]').click()

            window_after = browser.window_handles[1]
            browser.switch_to.window(window_after)

            # page5 = open('page5.html','w')
            # page5.write(browser.page_source)
            # page5.close()

            str_browser_dados = browser.page_source
            #print(str_browser_dados)
    except:
        continue
    break
    
  #--------------------------------------------------------------------#
    lista_dados.append(get_data_from_str(str_browser_dados))

pickle.dump(lista_dados, open( "lista_dados.pckl", "wb" ))





  0%|          | 0/94 [00:00<?, ?it/s]



  1%|          | 1/94 [00:22<35:19, 22.79s/it]



  2%|▏         | 2/94 [00:30<28:00, 18.27s/it]



  3%|▎         | 3/94 [00:38<22:52, 15.08s/it]



  4%|▍         | 4/94 [00:45<19:15, 12.84s/it]



  5%|▌         | 5/94 [01:00<19:53, 13.41s/it]



  6%|▋         | 6/94 [01:24<24:08, 16.46s/it]



  7%|▋         | 7/94 [01:31<20:06, 13.87s/it]



  9%|▊         | 8/94 [01:39<17:08, 11.96s/it]



 10%|▉         | 9/94 [02:29<33:01, 23.32s/it]



 11%|█         | 10/94 [02:36<25:56, 18.53s/it]



 12%|█▏        | 11/94 [02:43<20:52, 15.09s/it]



 13%|█▎        | 12/94 [02:51<17:34, 12.86s/it]



 14%|█▍        | 13/94 [03:19<23:39, 17.52s/it]



 15%|█▍        | 14/94 [03:26<19:11, 14.39s/it]



 16%|█▌        | 15/94 [03:34<16:08, 12.26s/it]



 17%|█▋        | 16/94 [03:41<14:06, 10.86s/it]



 18%|█▊        | 17/94 [03:48<12:33,  9.79s/it]



 19%|█▉        | 18/94 [03:56<11:33,  9.13s/it]



 20%|██        | 19/94 [04:25<18:56, 15.15s/i

In [0]:
str_browser_dados

'<html><head><meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">\n\n<title>Estação - 83236</title>\n   <!-- FACE BOX -->\n   <script src="facebox/js/jquery.js" type="text/javascript"></script>\n   <link href="facebox/styles/facebox.css" media="screen" rel="stylesheet" type="text/css">\n\n   <script src="facebox/js/facebox.js" type="text/javascript"></script>\n   <script type="text/javascript">\n      jQuery(document).ready(function($) {\n         $(\'a[rel*=facebox]\').facebox({\n            loading_image : \'loading.gif\',\n            close_image   : \'closelabel.gif\'})\n         $(\'area[rel*=facebox]\').facebox({\n            loading_image : \'loading.gif\',\n            close_image   : \'closelabel.gif\'})\n      })\n   </script>\n   <script type="text/javascript">\n      function showFoto(src) {\n         //criando um link falso, para o LightBox captar as informações\n         var a_false = document.createElement("A");\n         a_false.rel = "lightbox";\n  

In [0]:
dic['Dados'].head()

,Estacao,Data,Hora,VelocidadeVentoMedia,VelocidadeVentoMaximaMedia,EvaporacaoPiche,EvapoBHPotencial,EvapoBHReal,InsolacaoTotal,NebulosidadeMedia,NumDiasPrecipitacao,PrecipitacaoTotal,PressaoNivelMarMedia,PressaoMedia,TempMaximaMedia,TempCompensadaMedia,TempMinimaMedia,UmidadeRelativaMedia,VisibilidadeMedia,Unnamed: 19
0,82294,31/01/1999,0,3.989130,8.0,167.9,NaN,NaN,211.9,5.516129,8.0,26.9,NaN,NaN,31.977419,27.487742,23.587097,74.193548,NaN,NaN
1,82294,28/02/1999,0,3.336145,6.8,114.4,NaN,NaN,151.7,6.869048,12.0,123.0,NaN,NaN,31.325000,27.179286,23.671429,78.616071,NaN,NaN
2,82294,31/03/1999,0,2.262366,8.1,65.8,NaN,NaN,128.0,7.741935,28.0,402.8,NaN,NaN,29.890323,26.220000,23.170968,85.951613,NaN,NaN
3,82294,30/04/1999,0,2.724444,6.3,70.4,NaN,NaN,135.8,7.011111,24.0,143.0,NaN,NaN,30.436667,26.369333,23.056667,85.300000,NaN,NaN
4,82294,31/05/1999,0,2.046237,5.6,58.2,NaN,NaN,168.9,6.720430,25.0,204.4,NaN,NaN,30.438710,26.140645,23.038710,85.693548,NaN,NaN
